Обучаем модель с нелинейной архитектурой по базе HH, задаем кастомную сигнатуру для TensorFlow Serving. Создаем две версии модели, для работы с базой по москве и остальным городам
Для этого поработаем с базой резюме с HeadHunter и обучим нейронную сеть для решения задачи оценки зарплаты пользователя по указанным данным.  


# Start. Загрузка библиотек, датасета и предобработка данных

## Импорт библиотек

In [1]:
# Модуль для работы с операционной системой
import os

# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Библиотека tensorflow
import tensorflow as tf

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для создания модели нейронной сети
from tensorflow.keras.models import Model

# Слои для создания нейронной сети
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input, concatenate

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Разделение данных на выборки
from sklearn.model_selection import train_test_split

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

2023-03-13 12:37:01.840241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 12:37:02.019549: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 12:37:02.019565: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 12:37:02.837372: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Загрузка и подготовка данных

In [2]:
# скачиваем базу
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/hh_fixed.csv', None, quiet=True)

# Чтение файла базы данных
df = pd.read_csv('hh_fixed.csv', index_col=0)

# Вывод количества резюме и числа признаков
print(df.shape)

df.head()

(62967, 12)


,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано
3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,Имеется собственный автомобиль
4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,Не указано


In [3]:
# Настройка номеров столбцов

COL_SEX_AGE     = df.columns.get_loc('Пол, возраст')
COL_SALARY      = df.columns.get_loc('ЗП')
COL_POS_SEEK    = df.columns.get_loc('Ищет работу на должность:')
COL_POS_PREV    = df.columns.get_loc('Последеняя/нынешняя должность')
COL_CITY        = df.columns.get_loc('Город')
COL_EMPL        = df.columns.get_loc('Занятость')
COL_SCHED       = df.columns.get_loc('График')
COL_EXP         = df.columns.get_loc('Опыт (двойное нажатие для полной версии)')
COL_EDU         = df.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = df.columns.get_loc('Обновление резюме')

### Преобразование числовых данных

In [4]:
### Параметрические данные для функций разбора ###

# Курсы валют для зарплат
currency_rate = {'usd'    : 65.,
                 'kzt'    : 0.17,
                 'грн'    : 2.6,
                 'белруб' : 30.5,
                 'eur'    : 70.,
                 'kgs'    : 0.9,
                 'сум'    : 0.007,
                 'azn'    : 37.5
                }

# Списки и словари для разбиения на классы
# Для ускорения работы добавлен счетчик классов, который будет вычислен ниже

# Список порогов возраста
age_class = [0, [18, 23, 28, 33, 38, 43, 48, 53, 58, 63]]

# Список порогов опыта работы в месяцах
experience_class = [0, [7, 13, 25, 37, 61, 97, 121, 157, 193, 241]]

# Классы городов
city_class = [0, 
              {'москва'          : 0,
               'санкт-петербург' : 1,
               'новосибирск'     : 2,
               'екатеринбург'    : 2,
               'нижний новгород' : 2,
               'казань'          : 2,
               'челябинск'       : 2,
               'омск'            : 2,
               'самара'          : 2,
               'ростов-на-дону'  : 2,
               'уфа'             : 2,
               'красноярск'      : 2,
               'пермь'           : 2,
               'воронеж'         : 2,
               'волгоград'       : 2,
               'прочие города'   : 3
              }]

# Классы занятости
employment_class = [0, 
                    {'стажировка'          : 0,
                     'частичная занятость' : 1,
                     'проектная работа'    : 2,
                     'полная занятость'    : 3
                    }]

# Классы графика работы
schedule_class = [0, 
                  {'гибкий график'         : 0,
                   'полный день'           : 1,
                   'сменный график'        : 2,
                   'удаленная работа'      : 3
                  }]

# Классы образования
education_class = [0,
                   {'высшее образование'   : 0,
                    'higher education'     : 0,
                    'среднее специальное'  : 1,
                    'неоконченное высшее'  : 2,
                    'среднее образование'  : 3
                   }]

In [5]:
# Вычисление счетчиков для данных разбиения

for class_desc in [age_class,
                   experience_class,
                   city_class,
                   employment_class,
                   schedule_class,
                   education_class]:
    if isinstance(class_desc[1], list):
        class_desc[0] = len(class_desc[1]) + 1
    else:
        class_desc[0] = max(class_desc[1].values()) + 1

In [6]:
 # Получение one hot encoding представления значения класса
 
 def int_to_ohe(arg, class_list):
  
    # Определение размерности выходного вектора
    num_classes = class_list[0]

    # Поиск верного интервала для входного значения
    for i in range(num_classes - 1):
        if arg < class_list[1][i]:
            cls = i                       # Интервал найден - назначение класса
            break
    else:                                 # Внимание: for/else
        cls = num_classes - 1             # Интервал не найден - последний класс

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(cls, num_classes)

In [7]:
# Общая функция преобразования строки к multi-вектору
# На входе данные и словарь сопоставления подстрок классам

def str_to_multi(arg, class_dict):
    # Определение размерности выходного вектора
    num_classes = class_dict[0]

    # Создание нулевого вектора
    result = np.zeros(num_classes)
    
    # Поиск значения в словаре и, если найдено,
    # выставление 1. на нужной позиции
    for value, cls in class_dict[1].items():
        if value in arg:
            result[cls] = 1.

    return result

In [8]:
# Разбор значений пола, возраста

base_update_year = 2019

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    sex = 1. if 'муж' in arg else 0.

    try:
        # Выделение года и вычисление возраста
        years = base_update_year - int(re.search(r'\d{4}', arg)[0])

    except (IndexError, TypeError, ValueError):
        # В случае ошибки год равен 0
        years = 0

    return sex, years

In [9]:
# Преобразование значения возраста в one hot encoding

def age_years_to_ohe(arg):
    return int_to_ohe(arg, age_class)

In [10]:
# Преобразование данных об опыте работы в one hot encoding

def experience_months_to_ohe(arg):
    return int_to_ohe(arg, experience_class)

In [11]:
# Разбор значения зарплаты

def extract_salary(arg):
    try:
        # Выделение числа и преобразование к float
        value = float(re.search(r'\d+', arg)[0])

        # Поиск символа валюты в строке, и, если найдено,
        # приведение к рублю по курсу валюты
        for currency, rate in currency_rate.items():
            if currency in arg:
                value *= rate
                break

    except TypeError:
        # Если не получилось выделить число - вернуть 0
        value = 0.

    return value / 1000.                  # В тысячах рублей

In [12]:
# Разбор данных о городe и преобразование в one hot encoding

def extract_city_to_ohe(arg):
    # Определение размерности выходного вектора
    num_classes = city_class[0]

    # Разбивка на слова
    split_array = re.split(r'[ ,.:()?!]', arg)
    city = split_array[0]  # Берем первое слово строки - это и есть город
    
    # Поиск города в словаре и присвоение класса
    city_cls = city_class[1].get(city.lower(), -1)
    
    # Если город не в city_class - значит его класс "прочие города"
    if city_cls < 0:   
        city_cls = num_classes - 1

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(city_cls, num_classes)

In [13]:
# Разбор данных о желаемой занятости и преобразование в multi

def extract_employment_to_multi(arg):
    return str_to_multi(arg, employment_class)

In [14]:
# Разбор данных о желаемом графике работы и преобразование в multi

def extract_schedule_to_multi(arg):
    return str_to_multi(arg, schedule_class)

In [15]:
# Разбор данных об образовании и преобразование в multi

def extract_education_to_multi(arg):
    result = str_to_multi(arg, education_class)
    
    # Поправка: неоконченное высшее не может быть одновременно с высшим
    if result[2] > 0.:
        result[0] = 0.
    
    return result

In [16]:
# Разбор данных об опыте работы - результат в месяцах

def extract_experience_months(arg):
    try:
        # Выделение количества лет, преобразование в int
        years = int(re.search(r'(\d+)\s+(год.?|лет)', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество лет равно 0
        years = 0
    
    try:
        # Выделение количества месяцев, преобразование в int
        months = int(re.search(r'(\d+)\s+месяц', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество месяцев равно 0
        months = 0

    # Возврат результата в месяцах
    return years * 12 + months

Функции подготовки выборок

In [33]:
def extract_row_data(row):
  
    # Извлечение и преобразование данных
    sex, age = extract_sex_age_years(row[COL_SEX_AGE])      # Пол, возраст
    sex_vec = np.array([sex])                               # Пол в виде вектора
    age_ohe = age_years_to_ohe(age)                         # Возраст в one hot encoding
    city_ohe = extract_city_to_ohe(row[COL_CITY])           # Город
    empl_multi = extract_employment_to_multi(row[COL_EMPL]) # Тип занятости
    sсhed_multi = extract_schedule_to_multi(row[COL_SCHED]) # График работы
    edu_multi = extract_education_to_multi(row[COL_EDU])    # Образование
    exp_months = extract_experience_months(row[COL_EXP])    # Опыт работы в месяцах
    exp_ohe = experience_months_to_ohe(exp_months)          # Опыт работы в one hot encoding
    salary = extract_salary(row[COL_SALARY])                # Зарплата в тысячах рублей
    salary_vec = np.array([salary])                         # Зарплата в виде вектора

    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([sex_vec,
                        age_ohe, 
                        city_ohe,
                        empl_multi,
                        sсhed_multi,
                        edu_multi,
                        exp_ohe])
    
    # Возврат входных данных и выходных (зарплаты)
    return x_data, salary_vec


# Создание общей выборки
def construct_train_data(row_list):
    x_data = []
    y_data = []
    
    for row in row_list:
        x, y = extract_row_data(row)
        if y[0] > 0:                      # Данные добавляются, только если есть зарплата
            x_data.append(x)
            y_data.append(y)

    return np.array(x_data), np.array(y_data)

In [34]:
df['Город'][:5]

0    Новороссийск , готов к переезду (Анапа, Геленд...
1    Новосибирск ,  м. Березовая роща , не готов к ...
2    Москва , готов к переезду , готов к редким ком...
3    Москва ,  м. Щукинская , не готов к переезду ,...
4    Москва ,  м. Юго-Западная , не готов к переезд...
Name: Город, dtype: object

### !!! ВАЖНО!!! проверям и фиксируем "московскую" колонку

In [35]:
# Формирование выборки из загруженного набора данных    
x, y = construct_train_data(df.values)

In [36]:
x[:5,12]

array([0., 0., 1., 1., 1.])

In [37]:
# Форма наборов параметров и зарплат
print(x.shape) 
print(y.shape)

# Пример обработанных данных
n = 0 
print(x[n])
print(y[n])

(62967, 39)
(62967, 1)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[40.]


### Преобразование текстовых данных

#### Подготовка текстовых данных "Должность"

In [38]:
# Функция извлечения данных о профессии

def extract_prof_text(row_list):
    result = []
    
    # Для всех строк таблицы: собрать значения 
    # столбцов желаемой и прошлой должности
    # если есть информация о зарплате
    
    for row in row_list:
        if extract_salary(row[COL_SALARY]) > 0:
            result.append(str(row[COL_POS_SEEK]) + ' ' + str(row[COL_POS_PREV]))
    
    # Возврат в виде массива
    return result

In [39]:
# Извлечение текстов о профессии для выборки
prof_text = extract_prof_text(df.values) 

# Пример текста о профессии из резюме
print(df.values[120])
print(prof_text[120]) 

['Мужчина ,  33 года , родился 8 октября 1985' '250000 руб.'
 'Руководитель BI' 'Москва , не готов к переезду , готов к командировкам'
 'полная занятость' 'полный день'
 'Опыт работы 11 лет 6 месяцев  Руководитель BI 250 000 руб. Информационные технологии, интернет, телеком Инженер Аналитик Занятость: полная занятость График работы: полный день Опыт работы 11 лет 6 месяцев Декабрь 2017 — по настоящее время 1 год 5 месяцев ЭркаФарм Россия , erkapharm.com Медицина, фармацевтика, аптеки ... Аптека, оптика Руководитель отдела аналитики Стратегическая цель: создание единой корпоративной автоматизированной информационно-аналитической системы бизнес анализа; MDM: ведение и поддержка в актуальном состоянии мастер-справочников компании; разработка новых справочников\\атрибутов для обеспечения деятельности подразделений компании; контроль бизнес-процессов в части, касающейся мастер-данных; написание документов, закрепляющих регламентные нормы по ведению мастер-справочников; ETL: загрузка данных 

In [40]:
# Преобразование текстовых данных в числовые/векторные для обучения нейросетью

# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
prof_tokenizer = Tokenizer(num_words=3000,                                       # Объем словаря
                           filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # Убираемые из текста ненужные символы
                           lower=True,                                           # Приведение слов к нижнему регистру
                           split=' ',                                            # Разделитель слов
                           oov_token='unknown',                                  # Токен для слов, которые не вошли в словарь 
                           char_level=False)                                     # Указание разделять по словам, а не по единичным символам

prof_tokenizer.fit_on_texts(prof_text) # Обучаем токенайзер

In [41]:
# Преобразование текстов в последовательность индексов согласно частотному словарю
prof_seq = prof_tokenizer.texts_to_sequences(prof_text)

In [42]:
# Преобразование последовательностей индексов в bag of words
x_prof = prof_tokenizer.sequences_to_matrix(prof_seq)

In [43]:
# Проверка результата
print(x_prof.shape)

(62967, 3000)


#### Подготовка текстовых данных "Опыт работы"

In [44]:
# Функция извлечения данных описания опыта работы

def extract_exp_text(row_list):
    result = []
    
    # Для всех строк таблицы: собрать значения опыта работы,
    # если есть информация о зарплате
    for row in row_list:
        if extract_salary(row[COL_SALARY]) > 0:
            result.append(str(row[COL_EXP]))
    
    # Возврат в виде массива
    return result

In [45]:
# Извлечение текстов об опыте работы для выборки
exp_text = extract_exp_text(df.values) 

# Пример текста об опыте работы из резюме
print(df.values[120])
print(exp_text[120]) 

['Мужчина ,  33 года , родился 8 октября 1985' '250000 руб.'
 'Руководитель BI' 'Москва , не готов к переезду , готов к командировкам'
 'полная занятость' 'полный день'
 'Опыт работы 11 лет 6 месяцев  Руководитель BI 250 000 руб. Информационные технологии, интернет, телеком Инженер Аналитик Занятость: полная занятость График работы: полный день Опыт работы 11 лет 6 месяцев Декабрь 2017 — по настоящее время 1 год 5 месяцев ЭркаФарм Россия , erkapharm.com Медицина, фармацевтика, аптеки ... Аптека, оптика Руководитель отдела аналитики Стратегическая цель: создание единой корпоративной автоматизированной информационно-аналитической системы бизнес анализа; MDM: ведение и поддержка в актуальном состоянии мастер-справочников компании; разработка новых справочников\\атрибутов для обеспечения деятельности подразделений компании; контроль бизнес-процессов в части, касающейся мастер-данных; написание документов, закрепляющих регламентные нормы по ведению мастер-справочников; ETL: загрузка данных 

In [46]:
# Преобразование текстовых данных в числовые/векторные для обучения нейросетью

# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
exp_tokenizer = Tokenizer(num_words=3000,                                       # Объем словаря
                          filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # Убираемые из текста ненужные символы
                          lower=True,                                           # Приведение слов к нижнему регистру
                          split=' ',                                            # Разделитель слов
                          oov_token='unknown',                                  # Токен для слов, которые не вошли в словарь 
                          char_level=False)                                     # Указание разделять по словам, а не по единичным символам

exp_tokenizer.fit_on_texts(exp_text) # Обучаем токенайзер

In [50]:
# Преобразование текстов в последовательность индексов согласно частотному словарю
exp_seq = exp_tokenizer.texts_to_sequences(exp_text)

In [52]:
# Преобразование последовательностей индексов в bag of words
x_exp = exp_tokenizer.sequences_to_matrix(exp_seq)

In [53]:
# Проверка результата
print(x_exp.shape)

(62967, 3000)


### Формирование выборок. Разбиение данных на резюме Москвы и остальных городов

In [54]:
x_moscow = x[x[:, 12] == 1.0]
x_other = x[x[:, 12] == 0.0]

# Посмотрим на размеры массивов
print(x_moscow.shape)
print(x_other.shape)

x_moscow_prof = x_prof[x[:, 12] == 1.0]
x_other_prof = x_prof[x[:, 12] == 0.0]

# Посмотрим на размеры массивов
print(x_moscow_prof.shape)
print(x_other_prof.shape)

x_moscow_exp = x_exp[x[:, 12] == 1.0]
x_other_exp = x_exp[x[:, 12] == 0.0]

# Посмотрим на размеры массивов
print(x_moscow_exp.shape)
print(x_other_exp.shape)

y_moscow = y[x[:, 12] == 1.0]
y_other = y[x[:, 12] == 0.0]

# Посмотрим на размеры массивов
print(y_moscow.shape)
print(y_other.shape)

(23741, 39)
(39226, 39)
(23741, 3000)
(39226, 3000)
(23741, 3000)
(39226, 3000)
(23741, 1)
(39226, 1)


In [55]:
# Удалим лишние массивы, чтобы освободить память
del x, y, x_prof, x_exp

In [57]:
x_moscow_train, x_moscow_test, x_moscow_train_prof, x_moscow_test_prof, \
x_moscow_train_exp, x_moscow_test_exp, y_moscow_train, y_moscow_test = train_test_split(x_moscow,
                                                                                        x_moscow_prof,
                                                                                        x_moscow_exp,
                                                                                        y_moscow,
                                                                                        test_size=0.1,
                                                                                        random_state=42)

# Посмотрим на размеры массивов с данными
print(x_moscow_train.shape)
print(x_moscow_train_prof.shape)
print(x_moscow_train_exp.shape)
print(y_moscow_train.shape)
print(x_moscow_test.shape)
print(x_moscow_test_prof.shape)
print(x_moscow_test_exp.shape)
print(y_moscow_test.shape)

(21366, 39)
(21366, 3000)
(21366, 3000)
(21366, 1)
(2375, 39)
(2375, 3000)
(2375, 3000)
(2375, 1)


Сформируем обучающую и проверочную выборки для базе остальным городам.

In [58]:
x_other_train, x_other_test, x_other_train_prof, x_other_test_prof, \
x_other_train_exp, x_other_test_exp, y_other_train, y_other_test = train_test_split(x_other,
                                                                                    x_other_prof,
                                                                                    x_other_exp,
                                                                                    y_other,
                                                                                    test_size=0.1,
                                                                                    random_state=42)

# Посмотрим на размеры массивов с данными
print(x_other_train.shape)
print(x_other_train_prof.shape)
print(x_other_train_exp.shape)
print(y_other_train.shape)
print(x_other_test.shape)
print(x_other_test_prof.shape)
print(x_other_test_exp.shape)
print(y_other_test.shape)

(35303, 39)
(35303, 3000)
(35303, 3000)
(35303, 1)
(3923, 39)
(3923, 3000)
(3923, 3000)
(3923, 1)


### Нормализация данных

In [59]:
# Для нормализации данных используется готовый инструмент
y_moscow_scaler = StandardScaler()
y_other_scaler = StandardScaler()

# Нормализация выходных данных обучающих выборок
y_moscow_train_scaled = y_moscow_scaler.fit_transform(y_moscow_train)
y_other_train_scaled = y_other_scaler.fit_transform(y_other_train)

# Нормализация выходных данных тестовой выборки
y_moscow_test_scaled = y_moscow_scaler.transform(y_moscow_test)
y_other_test_scaled = y_other_scaler.transform(y_other_test)

# Посмотрим размеры нормализованных массивов
print(y_moscow_train_scaled.shape)
print(y_moscow_test_scaled.shape)
print(y_other_train_scaled.shape)
print(y_other_test_scaled.shape)

(21366, 1)
(2375, 1)
(35303, 1)
(3923, 1)


# Creat and fit model.
создаем и обучаем модели

## Московская сеть

In [60]:
m_input1 = Input((x_moscow_train.shape[1],))       # Вход нейронной сети для числовых данных
m_input2 = Input((x_moscow_train_prof.shape[1],))  # Вход нейронной сети для текстовых данных "должность"
m_input3 = Input((x_moscow_train_exp.shape[1],))   # Вход нейронной сети для текстовых данных "опыт работы"

m_x1 = m_input1                               # Ветка 1
m_x1 = Dense(128, activation="relu")(m_x1)
m_x1 = Dense(1000, activation="tanh")(m_x1)
m_x1 = Dense(100, activation="relu")(m_x1)

m_x2 = m_input2                               # Ветка 2
m_x2 = Dense(20, activation="relu")(m_x2)
m_x2 = Dense(500, activation="relu")(m_x2)
m_x2 = Dropout(0.3)(m_x2)

m_x3 = m_input3                               # Ветка 3
m_x3 = Dense(30, activation="relu")(m_x3)
m_x3 = Dense(800, activation="relu")(m_x3)
m_x3 = Dropout(0.3)(m_x3)

m_x = concatenate([m_x1, m_x2, m_x3])         # Объединение трех веток

m_x = Dense(15, activation='relu')(m_x)       # Промежуточный слой
m_x = Dropout(0.5)(m_x)
m_x = Dense(1, activation='linear')(m_x)      # Финальный регрессирующий нейрон

# Создаем модель нейронный сети при помощи класса Model
model_moscow = Model((m_input1, m_input2, m_input3), m_x)

2023-03-13 12:52:43.160064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 12:52:43.160279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 12:52:43.250295: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-13 12:52:43.250685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-13 12:52:43.250778: W tensorflow/c

In [61]:
model_moscow.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3000)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 3000)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          5120        ['input_1[0][0]']                
                                                                                              

In [62]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-03-13 12:53:08.532649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 12:53:08.532777: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False

In [63]:
# Компилируем модель
model_moscow.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Обучаем модель
history = model_moscow.fit([x_moscow_train, x_moscow_train_prof, x_moscow_train_exp],
                           y_moscow_train_scaled,
                           batch_size=128,
                           epochs=30,
                           validation_data=([x_moscow_test, x_moscow_test_prof, x_moscow_test_exp], y_moscow_test_scaled),
                           shuffle=True,
                           verbose=1)

2023-03-13 12:53:12.120434: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 256392000 exceeds 10% of free system memory.


Epoch 1/30


2023-03-13 12:53:13.007119: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 256392000 exceeds 10% of free system memory.


164/167 [============================>.] - ETA: 0s - loss: 0.6565 - mae: 0.4941

2023-03-13 12:53:15.621723: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 28500000 exceeds 10% of free system memory.
2023-03-13 12:53:15.642049: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 28500000 exceeds 10% of free system memory.


167/167 [==============================] - 2s 8ms/step - loss: 0.6524 - mae: 0.4931 - val_loss: 0.3751 - val_mae: 0.3999
Epoch 2/30
167/167 [==============================] - 1s 7ms/step - loss: 0.5273 - mae: 0.4232 - val_loss: 0.3116 - val_mae: 0.3524
Epoch 3/30
167/167 [==============================] - 1s 6ms/step - loss: 0.4482 - mae: 0.3965 - val_loss: 0.3412 - val_mae: 0.3589
Epoch 4/30
167/167 [==============================] - 1s 6ms/step - loss: 0.3802 - mae: 0.3764 - val_loss: 0.3406 - val_mae: 0.3825
Epoch 5/30
167/167 [==============================] - 1s 7ms/step - loss: 0.3303 - mae: 0.3636 - val_loss: 0.3033 - val_mae: 0.3415
Epoch 6/30
167/167 [==============================] - 1s 7ms/step - loss: 0.3266 - mae: 0.3492 - val_loss: 0.2968 - val_mae: 0.3359
Epoch 7/30
167/167 [==============================] - 1s 7ms/step - loss: 0.4382 - mae: 0.3511 - val_loss: 0.3089 - val_mae: 0.3470
Epoch 8/30
167/167 [==============================] - 1s 6ms/step - loss: 0.2910 - mae:

## Для остальных городов

In [67]:
o_input1 = Input((x_other_train.shape[1],))       # Вход нейронной сети для числовых данных
o_input2 = Input((x_other_train_prof.shape[1],))  # Вход нейронной сети для текстовых данных "должность"
o_input3 = Input((x_other_train_exp.shape[1],))   # Вход нейронной сети для текстовых данных "опыт работы"

o_x1 = o_input1                               # Ветка 1
o_x1 = Dense(128, activation="relu")(o_x1)
o_x1 = Dense(1000, activation="tanh")(o_x1)
o_x1 = Dense(100, activation="relu")(o_x1)

o_x2 = o_input2                               # Ветка 2
o_x2 = Dense(20, activation="relu")(o_x2)
o_x2 = Dense(500, activation="relu")(o_x2)
o_x2 = Dropout(0.3)(o_x2)

o_x3 = o_input3                               # Ветка 3
o_x3 = Dense(30, activation="relu")(o_x3)
o_x3 = Dense(800, activation="relu")(o_x3)
o_x3 = Dropout(0.3)(o_x3)

o_x = concatenate([o_x1, o_x2, o_x3])         # Объединение трех веток

o_x = Dense(15, activation='relu')(o_x)       # Промежуточный слой
o_x = Dropout(0.5)(o_x)
o_x = Dense(1, activation='linear')(o_x)      # Финальный регрессирующий нейрон

# Создаем модель нейронный сети при помощи класса Model
model_other = Model((o_input1, o_input2, o_input3), o_x)

In [68]:
# Компилируем модель
model_other.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Обучаем модель
history = model_other.fit([x_other_train, x_other_train_prof, x_other_train_exp],
                          y_other_train_scaled,
                          batch_size=128,

                          epochs=30,
                          validation_data=([x_other_test, x_other_test_prof, x_other_test_exp], y_other_test_scaled),
                          shuffle=True,
                          verbose=1)

2023-03-13 12:55:47.535712: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 423636000 exceeds 10% of free system memory.


Epoch 1/30
276/276 [==============================] - 3s 7ms/step - loss: 0.7328 - mae: 0.4567 - val_loss: 0.5158 - val_mae: 0.3846
Epoch 2/30
276/276 [==============================] - 2s 7ms/step - loss: 0.4954 - mae: 0.3773 - val_loss: 0.4493 - val_mae: 0.3417
Epoch 3/30
276/276 [==============================] - 2s 6ms/step - loss: 0.4196 - mae: 0.3523 - val_loss: 0.4367 - val_mae: 0.3303
Epoch 4/30
276/276 [==============================] - 2s 5ms/step - loss: 0.3876 - mae: 0.3366 - val_loss: 0.4208 - val_mae: 0.3209
Epoch 5/30
276/276 [==============================] - 2s 6ms/step - loss: 0.3336 - mae: 0.3188 - val_loss: 0.4588 - val_mae: 0.3171
Epoch 6/30
276/276 [==============================] - 1s 5ms/step - loss: 0.3201 - mae: 0.3087 - val_loss: 0.4274 - val_mae: 0.3145
Epoch 7/30
276/276 [==============================] - 1s 5ms/step - loss: 0.3744 - mae: 0.3156 - val_loss: 0.4002 - val_mae: 0.3059
Epoch 8/30
276/276 [==============================] - 2s 7ms/step - loss: 0.

# Задаем кастомную сигнатуру модели и сохраняем обученные НС.

## Moscow model

In [72]:
MODEL_DIR = './content/tmp/'                             # Путь к директории для хранения моделей

# Создание кастомной сигнатуры с помощью класса tf.Module
class MyModule(tf.Module):
    def __init__(self, model):  # При инициализации экземпляра класса передаем модели нейронной сети
        self.model = model      # Присваиваем модель в переменную
    
    @tf.function(input_signature=([tf.TensorSpec(shape=(None, x_other_train.shape[1]), dtype=tf.float32)], 
                                  [tf.TensorSpec(shape=(None, x_other_train_prof.shape[1]), dtype=tf.float32)],
                                  [tf.TensorSpec(shape=(None, x_other_train_exp.shape[1]), dtype=tf.float32)])) # Вызываем декоратор, указывая формы тензоров, которые будут поступать на вход метода.
    def score(self, input_1, input_2, input_3):
        result = self.model([input_1, input_2, input_3]) # Подаем данные на вход модели
        return {"result": result}                      # Возвращаем результат



In [73]:
module_moscow = MyModule(model_moscow) # Вызываем класс MyModule
# Задаем сигнатуру. 
# PS: tо есть указываем, что в качестве входов и выходов модели, которую будем загружать на сервер,\ 
# должны использоваться входы и выходы метода score.

signatures_moscow = {"score": module_moscow.score}

# Save model
version = 1                                            # Номер версии модели
model_moscow_path = os.path.join(MODEL_DIR, str(version))    # Путь к директории для хранения модели с заданной версией


tf.saved_model.save(module_moscow,                # Передаем модель, которую нужно сохранить
                    model_moscow_path,            # Путь для сохранения
                    signatures=signatures_moscow) # Указываем сигнатуру модели

print('Путь для экспорта: {}'.format(model_moscow_path))

INFO:tensorflow:Assets written to: ./content/tmp/1/assets


INFO:tensorflow:Assets written to: ./content/tmp/1/assets


Путь для экспорта: ./content/tmp/1


In [74]:
!saved_model_cli show --dir {model_moscow_path} --all

2023-03-13 13:01:49.708919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 13:01:49.834560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 13:01:49.834576: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 13:01:50.661477: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Other cities model

In [75]:
module_other = MyModule(model_other) # Вызываем класс MyModule # Вызываем класс MyModule
# Задаем сигнатуру. 
# PS: tо есть указываем, что в качестве входов и выходов модели, которую будем загружать на сервер,\ 
# должны использоваться входы и выходы метода score.

signatures_other = {"score": module_other.score}
# Save model
version = 2                                           # Номер версии модели
model_other_path = os.path.join(MODEL_DIR, str(version))    # Путь к директории для хранения модели с заданной версией


tf.saved_model.save(module_other,                # Передаем модель, которую нужно сохранить
                    model_other_path,            # Путь для сохранения
                    signatures=signatures_other) # Указываем сигнатуру модели

print('Путь для экспорта: {}'.format(model_other_path))

INFO:tensorflow:Assets written to: ./content/tmp/2/assets


INFO:tensorflow:Assets written to: ./content/tmp/2/assets


Путь для экспорта: ./content/tmp/2


In [76]:
!saved_model_cli show --dir {model_other_path} --all


2023-03-13 13:03:40.468396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 13:03:40.545081: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 13:03:40.545099: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 13:03:41.060966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Подготавливаем конфигурационный файл.
При использовании одной версии модели это необязательно

In [ ]:
!echo "" >> models.config

In [128]:
!ls

conf_file = """
model_config_list {
    config {
        name: 'regression_model'
        base_path: '/content/tmp/'
        model_platform: 'tensorflow'
        model_version_policy {
            specific {
                versions: 1
                versions: 2
            }
        }   
        version_labels {
            key: 'moscow'
            value: 1
        }
        version_labels {
            key: 'other'
            value: 2
        }
    }
}
"""

with open ('./content/models.config', 'w') as f:
    f.write(conf_file)

content		 serving
education.ipynb  tensorflow-serving_cifar10.ipynb
hh_fixed.csv	 test_sample.npy
predict.json	 tf_serving_hh_custom_2versions.ipynb
server.log	 tf_serving_hh_custom.ipynb


# Запуск tf-serving

## запуск на локальной машине. Исполнить команты в терминале. 
Ошибки в путях, позже разобрать

In [133]:
name_of_container = 'tf_serving_hh_2versions'
port = '8501:8501'
source = '/home/nikmih/Jupyter_projects/tensorflow_serving/content/'
conf_file_source = '/home/nikmih/Jupyter_projects/tensorflow_serving/content/models.config'
target = '/models/regression_model'
conf_file_target = '/models/models.config'
model_name = 'regression_model'
container_name = 'hh_container'
model_config_file='/models.config'
docker_run_comand = f'docker run -p {port} --name {container_name} \
--mount type=bind,source={source},target={target} \
--mount type=bind,source={conf_file_source},target={conf_file_target} \
-e MODEL_NAME={model_name} -t tensorflow/serving --model_config_file=/models/models.config \
--allow_version_labels_for_unavailable_models=true'

print(docker_run_comand)

'''
#containes list
docker sm
# docker stop container
docker stop 70f91474afac
# удалить контейнер
docker rm 
--mount type=bind,source={conf_file_source},target={conf_file_target} \
--model_config_file= {model_config_file}
'''

docker run -p 8501:8501 --name hh_container --mount type=bind,source=/home/nikmih/Jupyter_projects/tensorflow_serving/content/,target=/models/regression_model --mount type=bind,source=/home/nikmih/Jupyter_projects/tensorflow_serving/content/models.config,target=/models/models.config -e MODEL_NAME=regression_model -t tensorflow/serving --model_config_file=/models/models.config --allow_version_labels_for_unavailable_models=true


'\n#containes list\ndocker sm\n# docker stop container\ndocker stop 70f91474afac\n# удалить контейнер\ndocker rm \n--mount type=bind,source={conf_file_source},target={conf_file_target} --model_config_file= {model_config_file}\n'

## Запуск tfs в colabe

In [134]:
# Записываем путь к модели в качестве системной переменной
os.environ["MODEL_DIR"] = MODEL_DIR

In [135]:
# Задаем параметры запуска сервера, и запускаем обслуживание модели с файлом конфигурации
%%bash --bg 
nohup tensorflow_model_server \
    --rest_api_port=8501 \
    --allow_version_labels_for_unavailable_models=true \
    --model_config_file=models.config \
    --model_config_file_poll_wait_seconds=60 > server.log 2>&1

SyntaxError: invalid syntax (1249827391.py, line 3)

# Проверка вывода запроса

## Отправим POST запрос через requests

In [85]:
import requests # Модуль для составления HTTP-запросов
import json

In [116]:
def send_data(input_x, input_prof, input_exp):
    """
    Parameters:
    input_x       - числовые данные из резюме
    input_prof    - текстовые данные о должности
    input_exp     - текстовые данные об опыте работы

    Return:
    unscaled_pred - предсказанная цена в исходном диапазоне значений
    """

    # Добавим к входным данным размерность для батча
    input_x = np.expand_dims(input_x, axis=0)
    input_prof = np.expand_dims(input_prof, axis=0)
    input_exp = np.expand_dims(input_exp, axis=0)

    # Передаем данные для записи в формат JSON
    data = json.dumps({"signature_name": "score", "inputs": {"input_1": input_x.tolist(),
                                                             "input_2": input_prof.tolist(),
                                                             "input_3": input_exp.tolist()}})
    
    # Указываем, что будем отправлять данные в формате JSON
    headers = {"content-type": "application/json"} 

    # Отправляем данные на сервер и получаем результат
    if input_x[0, 12] == 1.0:                                         # Если в данных записан город Москва, то отправляем в модель для Москвы
        json_response = requests.post('http://localhost:8501/v1/models/regression_model/versions/1:predict', 
                                      data=data, 
                                      headers=headers)
        prediction = json.loads(json_response.text)['outputs']        # Извлекаем предсказание
        unscaled_pred = y_other_scaler.inverse_transform(prediction)

#         unscaled_pred = y_moscow_scaler.inverse_transform(prediction) # Делаем обратное преобразование цены при помощи скейлера для Москвы
    else:                                                             # Иначе отправляем во вторую версию модели
        json_response = requests.post('http://localhost:8501/v1/models/regression_model/labels/other:predict', 
                                      data=data, 
                                      headers=headers)
#         unscaled_pred = json.loads(json_response.text)
        prediction = json.loads(json_response.text)['outputs']        # Извлекаем предсказание
        unscaled_pred = y_other_scaler.inverse_transform(prediction)  # Делаем обратное преобразование цены при помощи скейлера для других городов

    return unscaled_pred # Возвращаем результат

## Checking moskow

In [120]:
n = 8
pred = send_data(x_moscow_test[n], x_moscow_test_prof[n], x_moscow_test_exp[n])
print('Реальное значение: {:6.2f}  Предсказанное значение: {:6.2f}  Разница: {:6.2f}'.format(y_moscow_test[n, 0],
                                                                                             pred[0, 0],
                                                                                             abs(y_moscow_test[n, 0] - pred[0, 0])))

{'error': 'Servable not found for request: Specific(regression_model, 1)'}

## Cheking other

In [121]:
n = 8
pred = send_data(x_other_test[n], x_other_test_prof[n], x_other_test_exp[n])
pred

{'error': 'Unrecognized servable version label: other'}